## Acknowledgements

This project uses the the method prensented by https://arxiv.org/abs/1904.12848.  

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import time
import argparse
from statistics import mean

import torch
from torch import nn
from torch.backends import cudnn

from torch.utils.tensorboard import SummaryWriter

from importnb import Notebook

import random

!pip install import-ipynb
import import_ipynb
from transforms import AddTransform

import create_datasets as dataset

with Notebook():
    import networks
    
from torch.utils.data import DataLoader, Subset, Dataset, ConcatDataset, TensorDataset
import numpy as np
#from RandAugment import RandAugment
from torchvision.transforms import Compose, ToTensor, Normalize, Pad, RandomCrop, RandomHorizontalFlip, RandomErasing

In [2]:
# Unsupervised Data Augmentation(UDA) (paper: https://arxiv.org/abs/1904.12848)
parser = argparse.ArgumentParser(description='Pythorch UDA CIFAR-10 implementation')
parser.add_argument('--evaluate', '-e', action='store_true', help='Evaluation mode')
parser.add_argument('--epochs', default=500, type=int, metavar='N',
                    help='number of total epochs to run')#200
# manually set the starting epoch, useful for resuming training
parser.add_argument('--start-epoch', '-se', default=0, type=int, metavar='N',
                    help='manual epoch number (useful on restarts)')
parser.add_argument('--save-dir', dest='save_dir',
                    help='The directory used to save the trained models',
                    default='save_2', type=str)
parser.add_argument('--resume', default='save_2', type=str, metavar='PATH',
                    help='path to latest checkpoint (default: save)')
parser.add_argument('--device', '-d', type=int, default=1, help='set cuda device')
args = parser.parse_args('')
best_prec1 = 0
device = args.device

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device}.")

Using cpu.


In [4]:
    TRAIN_PARAMETERS = {
    'num_labelled_indices': 2000}

In [5]:
train_transform_labelled_else = Compose([
    #Pad(4),
    #RandomCrop(32, fill=128),
    #RandomHorizontalFlip(),
    ToTensor(),
    Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    RandomErasing(scale=(0.1, 0.33)),
])

In [6]:
train_transform_labelled = Compose([
    #Pad(4),
    #RandomCrop(32, fill=128),
    #RandomHorizontalFlip(),
    #ToTensor(),
    Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
    RandomErasing(scale=(0.1, 0.33)),
])

In [7]:
test_transform = Compose([
    ToTensor(),
    Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

In [8]:
writer =  SummaryWriter('1000, 3,9')

def save_checkpoint(state, filename='checkpoint.pth.tar'):
    """
    Save the training model
    """
    torch.save(state, filename)

class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    #Top-k accuracy: how often the correct label is in the model's top-k predictions.
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []       
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res
            

def uda_train(train_labelled, train_unlabelled, train_unlabelled_aug, model, criterion, consistency_criterion, optimizer, epoch):
    """
        Run one train epoch
    """
    data_time = AverageMeter()
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()

    lam = 1.0
    #lam = 0.5
    
    label_iter = iter(train_labelled)
    unsup_iter = iter(train_unlabelled)

    # train over 1 epoch, drop y for unlabelled data
    for i, (unlabel_aug_x, _) in enumerate(train_unlabelled_aug):
        # measure data loading time
        data_time.update(time.time() - end)

        # SUPERVISED
        try:
            x,y = next(label_iter)
        except StopIteration:
            label_iter = iter(train_labelled)
            x, y = next(label_iter)

        x = x.to(device)
        y = y.to(device)
        #print('x.shape: ',x.shape)
        y_pred = model(x)

        sup_loss = criterion(y_pred, y)

        # UNSUPERVISED
        unlabel_x, _ = next(unsup_iter)
        unlabel_x = unlabel_x.to(device)
        unlabel_aug_x = unlabel_aug_x.to(device)

        unsup_y_pred = model(unlabel_x).detach()

        unsup_y_probas = torch.softmax(unsup_y_pred, dim=-1)

        unsup_aug_y_pred = model(unlabel_aug_x)
        unsup_aug_y_probas = torch.log_softmax(unsup_aug_y_pred, dim=-1)

        unsup_loss = consistency_criterion(unsup_aug_y_probas, unsup_y_probas)

        final_loss = sup_loss + lam * unsup_loss

        optimizer.zero_grad()
        final_loss.backward()
        optimizer.step()

        # measure accuracy and record loss
        losses.update(final_loss.item(), unlabel_aug_x.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % 50 == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
                epoch, i, len(train_unlabelled_aug), batch_time=batch_time,
                data_time=data_time, loss=losses))

    return losses.avg

def uda_validate(valid_loader, unlabelled_loader, model, criterion, epoch):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    
    print('valid_loader type', type(valid_loader))
    print('valid_loader len:', len(valid_loader))
    
    model.eval()

    end = time.time()

    with torch.no_grad():
        for i, (input, target) in enumerate(valid_loader):
            target = target.to(device)
            
            #print('target len', len(target))
            input_var = input.to(device)
            target_var = target.to(device)

            # compute output
            output = model(input_var)
            loss = criterion(output, target_var)
            
            output = output.float()
            loss = loss.float()  
            
            # measure accuracy and record loss
            prec1 = accuracy(output.data, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec1.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % 50 == 0:
                print('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                          i, len(valid_loader), batch_time=batch_time, loss=losses,
                          top1=top1))

    print(' * Prec@1 {top1.avg:.3f}'.format(top1=top1))

    return top1.avg, losses.avg
 
       
def create_dataset(unlabelled_subset_loader,unlabelled_subset,model,train_labelled_ds,criterion):
        #print('train_labelled_ds tzpe0:', type(train_labelled_ds[0]))
        #print('train_labelled_ds tzpe 0 0:', type(train_labelled_ds[0][0]))
     #   list_indices = range(200)
      #  new_data = Subset(train_unlabelled_ds, list_indices)
        
       # print('length of new_data: ', len(new_data))
       # print('length of new_data 0: ', len(new_data[0]))
        
        #print('new_data[0] len: ', new_data[0][0].size())
        
     #   print('unlabelled_subset_type:',type(unlabelled_subset))
     #   print('unlabelled_subset_type:',type(unlabelled_subset[0]))
      #  print('unlabelled_subset_targets:',unlabelled_subset[:][1])
     #   print('unlabelled_subset_targets:',unlabelled_subset[0][1])
     #   print('unlabelled_subset_targets:',unlabelled_subset[1][1])
      #  print('unlabelled_subset_targets:',unlabelled_subset[2][1])
        
        unlabelled_subset_l = [list(row) for row in unlabelled_subset]
        print('unlabelled_subset_l_len',len(unlabelled_subset_l))
      #  print('unlabelled_subset_l',unlabelled_subset_l[0])
        
        ###
        batch_time = AverageMeter()
        losses = AverageMeter()
        top1 = AverageMeter()
        top90 = AverageMeter()

        model.eval()
        idx_remove = []
        end = time.time()
        with torch.no_grad():
          #  x_temp = []
          #  y_temp = []
            for i, (input,target) in enumerate(unlabelled_subset_loader):
                target = target.to(device)
                print('target: ', target)
                #print('target shape: ', target.shape)
                target2 = target.clone().detach()
              #  print('type target'.type(target))
                
                inputs_var = input.to(device)
                target_var = target.to(device)

                # compute output
                output = model(inputs_var)
                #print('outputs: ',outputs)
                
                
                loss = criterion(output, target_var)
               # print('target_var:',target_var)
              #  print('loss:',loss)
                
            
                outputs = output.float()
                loss = loss.float()  
                outputs2 = outputs.clone().detach()
                #print('type outputs'.type(outputs))
                
                # measure accuracy and record loss
                prec1 = accuracy(outputs.data, target)[0]
                losses.update(loss.item(), input.size(0))
                top1.update(prec1.item(), input.size(0))

                # measure elapsed time
                batch_time.update(time.time() - end)
                end = time.time()

                #if i % 50 == 0:
                print('Testing 200 samples: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                          i, len(unlabelled_subset_loader), batch_time=batch_time, loss=losses,
                          top1=top1))              
            
                
               # softmax = torch.nn.Softmax(dim=1)
                output_relu = torch.nn.functional.relu(output)
                softmax = torch.nn.functional.softmax(output,dim=-1)
                output_softmax,top_class = softmax.topk(1,dim =1)
                c= output_softmax.cpu()
                b = c.numpy()
                a = np.where(b>=0.90)
                print('higher 90:',len(a[0]))
              #  print('out_softmax_len:', len(output_softmax))
             #   print('class:',top_class[a[0]])
               # output_softmax = torch.softmax(output_relu,dim=-1)
               # output_softmax = torch.exp(output)
                #output_softmax = torch.softmax(output, dim = 0)
                _,output_target = torch.max(output, 1)
              #  print('output:',output)
               # print('output_relu:',output_relu)
             #   print('output_softmax:',output_softmax)
              #  print('output_target',output_target)
           #     print('output_target_element',output_target.data[0])
           
                ####!!!works for number images smaller than 64
                
                #for j in range(len(output_target)):
                #    unlabelled_subset_l[i*64+j][1] = int(output_target.data[j])
               
            #    print('output_softmax.data[j]',output_softmax.data[0])    
                for j in range(len(output_target)):
                    if output_softmax.data[j]>0.90:
                        unlabelled_subset_l[i*64+j][1] = int(output_target.data[j])
                      
                    else:
                         idx_remove.append(i*64+j) 
                        # target2 i*64+j] 
              #  print('idx_remove:',idx_remove) 
              #  print('idx_remove[p]:',idx_remove[0]) 
            for ele in sorted(idx_remove,reverse=True):
                #for p in range(len(idx_remove)):
                unlabelled_subset_l.pop(ele)
                outputs2 = outputs2[torch.arange(outputs2.size(0))!=ele] 
                target2 = target2[torch.arange(target2.size(0))!=ele] 
                #    print('pop')
                
         #   print('output2',outputs2)
            print('unlabelled_subset_l length:',len(unlabelled_subset_l))
            prec90 = accuracy(outputs2.data, target2)[0]
            print('prec90',prec90)
            #print('mean',np.mean(prec90))
            top90.update(prec90.item(), input.size(0))
        print(' * Accuray@90 {top90.avg:.3f}'.format(top90=top90))
            
            #print('i:',i)
                
     #   print('unlabelled_subset_targets:',unlabelled_subset_l[0][1])
     #   print('unlabelled_subset_targets:',unlabelled_subset_l[1][1])
     #   print('unlabelled_subset_targets:',unlabelled_subset_l[2][1])
        
        
             
      #  unlabelled_subset_l[0] = [torch.tensor(np.asarray(row)) for row in unlabelled_subset_l[0]]
     #   print('unlabelled_subset_tup type:',type(unlabelled_subset_l))
     #   print('unlabelled_subset_tup type[0]:',type(unlabelled_subset_l[0]))
     #   print('unlabelled_subset_tup type[0][0]:',type(unlabelled_subset_l[0][0]))
     #   print('unlabelled_subset_tup type[0][1]:',type(unlabelled_subset_l[0][1]))
        
        
        ############
        #int for labels
        #tensor for images
        #+transform entspricht output aus addtransform
        #für beide datensätze transformieren
        
     #   unlabelled_subset_tup_tensor = torch.stack(unlabelled_subset_tup)
      #  unlabelled_subset_data = TensorDataset(unlabelled_subset_tup_tensor)
       # print('unlabelled_subset_data type:',type(unlabelled_subset_data))
        #create new labelled dataset
        
        #train_labelled_ds_new = ConcatDataset([train_labelled_ds,unlabelled_subset_l])
        train_labelled_ds_new = ConcatDataset([train_labelled_ds,unlabelled_subset_l])
            
       
        print(' * Prec@1 {top1.avg:.3f}'.format(top1=top1))
        
        return train_labelled_ds_new


def run_unsupervised():
    valacc = 0.
    global args, best_prec1
    best_valid_loss = 10e10

    # Check the save_dir exists or not
    if not os.path.exists(args.save_dir):
        os.makedirs(args.save_dir)

    # load model
    model = networks.FastResNet(num_classes=10).to(device)
    print(model) 

    # data loaders
    train_labelled_ds, train_unlabelled_ds, train_unlabelled_aug, test = dataset.cifar10_unsupervised_dataloaders(TRAIN_PARAMETERS['num_labelled_indices'])
    
  #  print('train_unlabelled_ds type:', type(train_unlabelled_ds))
  #  print('train_unlabelled_ds type:', type(train_unlabelled_ds[0]))
  #  print('train_unlabelled_ds type:', type(train_unlabelled_ds[0][0]))
   
    # print('test type :', type(test)
    
    # criterion and optimizer
    criterion = nn.CrossEntropyLoss().to(device)
    consistency_criterion = nn.KLDivLoss(reduction='batchmean').to(device)

    optimizer = torch.optim.SGD(model.parameters(),
                                lr=0.01,
                                momentum=0.9,
                                weight_decay=1e-4,
                                nesterov=True)

    # warmup steps
    t_max = len(train_labelled_ds) * args.epochs
    #t_max = len(train_labelled) * args.epochs
    
    eta_min = 0.03 * 0.004

    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=t_max, eta_min=eta_min)
    '''
    if args.resume:
        if os.path.isfile(args.resume):
            print("=> loading checkpoint '{}'".format(args.resume))
            checkpoint = torch.load(args.resume)
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            scheduler.load_state_dict(checkpoint['scheduler'])
            args.start_epoch = checkpoint['epoch']
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args.evaluate, checkpoint['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(args.resume))
        '''
    #cudnn.benchmark = True

    # UDA train loop
    count = 0
    count_else = 0
    num_labelled_indices_new =200
    train_labelled_ds_new = train_labelled_ds
    valacc_ref = np.arange(70,100,1)
   # print('valacc_ref:',valacc_ref)
   # print('valacc_ref_count:',valacc_ref[count])
    for epoch in range(args.start_epoch, args.epochs):
        #valacc =1
      # Hi =1
        if count > 29:
            count = 0
        if valacc >valacc_ref[count] or (count_else>3 and count!=0): # True: #valacc> 50.:
             #test(valacc, model, train_unlabelled_ds, new_data, optimizer, epoch)
            print('higher than 70')
          #  print('train_labelled_ds type :', type(train_labelled_ds))
          #  print('train_labelled_ds tzpe 0 0:', type(train_labelled_ds[0][0]))
            
           # indices = range(200)
           # new_data = torch.utils.data.SubsetRandomSampler(indices)
            
            count_else = 0
            labelled_indices = []
    
            #num_classes =10
            #indices = np.random.permutation(np.arange(0,(count+1)*num_labelled_indices_new))
            indices0 = np.random.permutation(40000)
          #  indices = indices0[0:(count+1)*num_labelled_indices_new]
            indices = indices0[0:num_labelled_indices_new]
            #print('indices_len:',len(indices))
            #print('indices:',indices)
           # class_counters = list([0] * num_classes)
           # max_counter = 10000 // num_classes

            for i in indices:
                labelled_indices.append(i)
                   

            #labelled_indices = random.sample(labelled_indices, (count+1)*num_labelled_indices_new)
            labelled_indices = random.sample(labelled_indices, num_labelled_indices_new)
            # Unlabelled subset
            unlabelled_subset_ds = Subset(train_unlabelled_ds, labelled_indices)
            
            #train_labelled_ds__new_t = AddTransform(train_labelled_ds_new, train_transform)
            unlabelled_subset_ds_t = AddTransform(unlabelled_subset_ds,train_transform_labelled_else)
           
            unlabelled_subset_loader = DataLoader(
            unlabelled_subset_ds_t,
            batch_size=64,
            shuffle=False,
            num_workers=8,
            pin_memory=True,
            )
            
            #new_data = random.sample (list(train_unlabelled_ds),200)
            
            print('unlabelled_subset_ds type :', type(unlabelled_subset_ds))
            print('unlabelled_subset_ds len :', len(unlabelled_subset_ds))
            
            #print('new_data type 0:', type(new_data[0]))
            
            
            #new_data_t = AddTransform(new_data, train_transform_labelled_else)
            #train_labelled_ds_t = AddTransform(train_labelled_ds,train_transform_labelled_else)
          
            #train_unlabelled_ds_1 = AddTransform(train_unlabelled_ds,test_transform)
        
          # new_data_loader = DataLoader(
           # train_unlabelled_ds_1,
           # batch_size=64,
          #  shuffle=False,
           # num_workers=8,
           # pin_memory=True,
           # sampler = new_data
           # )
           # train_labelled_ds_new = create_dataset(unlabelled_subset_loader, unlabelled_subset_ds,model,train_labelled_ds,criterion)
            train_labelled_ds_new = create_dataset(unlabelled_subset_loader, unlabelled_subset_ds,model,train_labelled_ds_new,criterion)
            count +=1
            print('count:',count)
             #train(valacc, model, train_unlabelled_ds, new_data, optimizer, epoch)
        else:      
            count_else+=1
            print('count_else:',count_else)
            #print('lower than 70')
            #train_labelled_ds_t = AddTransform(train_labelled_ds, train_transform_labelled_else)
            
        train_labelled_ds_t = AddTransform(train_labelled_ds_new, train_transform_labelled_else)
        print('train_labelled_ds_t_in_Datload len :', len(train_labelled_ds_t))
      #  print('train_labelled_ds_t_in_Datload type :', type(train_labelled_ds_t[0]))
      #  print('train_labelled_ds_t_in_Datload type [0][0]:', type(train_labelled_ds_t[0][0]))
      #  print('train_labelled_ds_t_in_Datload type :', type(train_labelled_ds_t[0][1]))
      #  print('train_labelled_ds_t_in_Datload type [0][0]:', train_labelled_ds_t[0][0])
      #  print('train_labelled_ds_t_in_Datload type [0][1]:', train_labelled_ds_t[0][1])
        
        
        train_unlabelled_ds_t = AddTransform(train_unlabelled_ds, train_transform_labelled_else)
        # Data loader for labeled and unlabeled train dataset
        train_labelled = DataLoader(
            train_labelled_ds_t,
            batch_size=64,
            shuffle=True,
            num_workers=8,
            pin_memory=True
        )
    
        train_unlabelled = DataLoader(
            train_unlabelled_ds_t,
            batch_size=64,
            shuffle=False,
            num_workers=8,
            pin_memory=True
        )

        
        trainloss = uda_train(train_labelled, train_unlabelled, train_unlabelled_aug, model, criterion, consistency_criterion, optimizer, epoch)

        writer.add_scalar('Loss/train', trainloss, epoch)

        scheduler.step()

        # evaluate on validation set

        #test = DataLoader(
        #train_labelled_ds_t, batch_size=64,
        #shuffle=False,
        #num_workers=8,
        #pin_memory=True
        #)
        
        valacc, valloss = uda_validate(test, train_unlabelled, model, criterion, epoch)

        is_best = valacc > best_prec1
        best_prec1 = max(valacc, best_prec1)

        writer.add_scalar('Acc/valid', valacc, epoch)
        writer.add_scalar('Loss/valid', valloss, epoch)
        
        
        # save checkpoint
        if epoch > 0 and epoch % 50 == 0:
            print('Save checkpoint')
            save_checkpoint({
                'epoch': epoch + 1,
                'scheduler': scheduler.state_dict(),
                'optimizer': optimizer.state_dict(),
                'state_dict': model.state_dict(),
                'best_prec1': best_prec1,
            }, filename=os.path.join(args.save_dir, 'checkpoint.th'))
        
        if (is_best):
            print('Saving better model')
            save_checkpoint({
                'epoch': epoch + 1,
                'scheduler': scheduler.state_dict(),
                'optimizer': optimizer.state_dict(),
                'state_dict': model.state_dict(),
                'best_prec1': best_prec1,
            }, filename=os.path.join(args.save_dir, 'best_model.th'))

            writer.add_scalar('Acc/valid_best', best_prec1, epoch)
            writer.add_scalar('Acc/los_best', valloss, epoch)

In [9]:
if __name__ == '__main__':
    run_unsupervised() 

FastResNet(
  (prep): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): GhostBatchNorm(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): CELU(alpha=0.3)
  )
  (layer1): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): GhostBatchNorm(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): CELU(alpha=0.3)
    (4): IdentityResidualBlock(
      (res1): Sequential(
        (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): GhostBatchNorm(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): CELU(alpha=0.3)
      )
      (res2): Sequential(
        (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (1): GhostBatchNorm(128, eps=1e-0

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x118e9f7e0>
Traceback (most recent call last):
  File "/Users/tayebeh/miniconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1618, in __del__
    self._shutdown_workers()
  File "/Users/tayebeh/miniconda3/lib/python3.12/site-packages/torch/utils/data/dataloader.py", line 1582, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/Users/tayebeh/miniconda3/lib/python3.12/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
          ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/tayebeh/miniconda3/lib/python3.12/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/tayebeh/miniconda3/lib/python3.12/multiprocessing/connection.py", line 1136, in wait
    ready = selector.select(timeout)
            ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/tayebeh/min

KeyboardInterrupt: 

Epoch: [38][200/688]	Time 0.034 (0.046)	Data 0.000 (0.006)	Loss 0.9890 (0.9875)	
Epoch: [38][250/688]	Time 0.033 (0.043)	Data 0.000 (0.005)	Loss 1.1693 (0.9886)	
Epoch: [38][300/688]	Time 0.033 (0.043)	Data 0.000 (0.004)	Loss 0.9424 (0.9920)	
Epoch: [38][350/688]	Time 0.033 (0.043)	Data 0.000 (0.003)	Loss 1.0110 (0.9884)	
Epoch: [38][400/688]	Time 0.033 (0.043)	Data 0.000 (0.003)	Loss 0.7865 (0.9887)	
Epoch: [38][450/688]	Time 0.054 (0.043)	Data 0.000 (0.003)	Loss 1.0530 (0.9864)	
Epoch: [38][500/688]	Time 0.033 (0.042)	Data 0.000 (0.002)	Loss 0.9287 (0.9868)	
Epoch: [38][550/688]	Time 0.033 (0.042)	Data 0.000 (0.002)	Loss 0.7728 (0.9866)	
Epoch: [38][600/688]	Time 0.033 (0.042)	Data 0.000 (0.002)	Loss 0.8299 (0.9870)	
Epoch: [38][650/688]	Time 0.033 (0.042)	Data 0.000 (0.002)	Loss 0.7587 (0.9923)	
valid_loader type <class 'torch.utils.data.dataloader.DataLoader'>
valid_loader len: 157
Test: [0/157]	Time 0.398 (0.398)	Loss 0.7552 (0.7552)	Prec@1 76.562 (76.562)
Test: [50/157]	Time 0.00

Epoch: [43][300/688]	Time 0.033 (0.043)	Data 0.000 (0.004)	Loss 0.8365 (0.9911)	
Epoch: [43][350/688]	Time 0.034 (0.043)	Data 0.000 (0.003)	Loss 0.8925 (0.9918)	
Epoch: [43][400/688]	Time 0.033 (0.043)	Data 0.000 (0.003)	Loss 0.9504 (0.9903)	
Epoch: [43][450/688]	Time 0.033 (0.043)	Data 0.000 (0.003)	Loss 0.9758 (0.9886)	
Epoch: [43][500/688]	Time 0.033 (0.042)	Data 0.000 (0.002)	Loss 0.8547 (0.9873)	
Epoch: [43][550/688]	Time 0.033 (0.042)	Data 0.000 (0.002)	Loss 0.9926 (0.9866)	
Epoch: [43][600/688]	Time 0.033 (0.042)	Data 0.000 (0.002)	Loss 0.9803 (0.9872)	
Epoch: [43][650/688]	Time 0.033 (0.042)	Data 0.000 (0.002)	Loss 0.9177 (0.9936)	
valid_loader type <class 'torch.utils.data.dataloader.DataLoader'>
valid_loader len: 157
Test: [0/157]	Time 0.402 (0.402)	Loss 0.7378 (0.7378)	Prec@1 81.250 (81.250)
Test: [50/157]	Time 0.005 (0.012)	Loss 0.6986 (0.7236)	Prec@1 73.438 (78.922)
Test: [100/157]	Time 0.004 (0.008)	Loss 0.6181 (0.6995)	Prec@1 78.125 (79.316)
Test: [150/157]	Time 0.004 (0

...

Test: [100/157]	Time 0.004 (0.008)	Loss 0.7101 (0.6839)	Prec@1 78.125 (78.636)
Test: [150/157]	Time 0.004 (0.007)	Loss 0.6094 (0.6851)	Prec@1 82.812 (78.704)
 * Prec@1 78.660
count_else: 478
train_labelled_ds_t_in_Datload len : 4000
Epoch: [477][0/688]	Time 1.190 (1.190)	Data 1.154 (1.154)	Loss 1.0816 (1.0816)	
Epoch: [477][50/688]	Time 0.033 (0.056)	Data 0.000 (0.023)	Loss 0.7506 (0.9324)	
Epoch: [477][100/688]	Time 0.033 (0.049)	Data 0.000 (0.012)	Loss 0.9762 (0.9263)	
Epoch: [477][150/688]	Time 0.033 (0.047)	Data 0.000 (0.008)	Loss 0.8897 (0.9186)	
Epoch: [477][200/688]	Time 0.033 (0.046)	Data 0.000 (0.006)	Loss 0.9881 (0.9174)	
Epoch: [477][250/688]	Time 0.033 (0.043)	Data 0.000 (0.005)	Loss 1.1298 (0.9167)	
Epoch: [477][300/688]	Time 0.033 (0.043)	Data 0.000 (0.004)	Loss 0.8417 (0.9166)	
Epoch: [477][350/688]	Time 0.033 (0.043)	Data 0.000 (0.003)	Loss 0.9222 (0.9170)	
Epoch: [477][400/688]	Time 0.033 (0.043)	Data 0.000 (0.003)	Loss 1.2055 (0.9170)	
Epoch: [477][450/688]	Time 0.033

Epoch: [482][0/688]	Time 1.216 (1.216)	Data 1.182 (1.182)	Loss 1.0332 (1.0332)	
Epoch: [482][50/688]	Time 0.033 (0.057)	Data 0.000 (0.023)	Loss 0.9471 (0.9350)	
Epoch: [482][100/688]	Time 0.033 (0.050)	Data 0.000 (0.012)	Loss 0.9244 (0.9369)	
Epoch: [482][150/688]	Time 0.034 (0.047)	Data 0.000 (0.008)	Loss 0.8568 (0.9269)	
Epoch: [482][200/688]	Time 0.033 (0.046)	Data 0.000 (0.006)	Loss 0.9537 (0.9282)	
Epoch: [482][250/688]	Time 0.033 (0.044)	Data 0.000 (0.005)	Loss 0.8576 (0.9280)	
Epoch: [482][300/688]	Time 0.033 (0.043)	Data 0.000 (0.004)	Loss 0.8169 (0.9266)	
Epoch: [482][350/688]	Time 0.033 (0.043)	Data 0.000 (0.003)	Loss 1.0738 (0.9218)	
Epoch: [482][400/688]	Time 0.033 (0.043)	Data 0.000 (0.003)	Loss 0.8754 (0.9167)	
Epoch: [482][450/688]	Time 0.034 (0.043)	Data 0.000 (0.003)	Loss 0.9256 (0.9177)	
Epoch: [482][500/688]	Time 0.033 (0.042)	Data 0.000 (0.002)	Loss 0.8334 (0.9153)	
Epoch: [482][550/688]	Time 0.033 (0.042)	Data 0.000 (0.002)	Loss 0.9549 (0.9159)	
Epoch: [482][600/68

Epoch: [487][100/688]	Time 0.033 (0.049)	Data 0.000 (0.011)	Loss 0.8982 (0.9222)	
Epoch: [487][150/688]	Time 0.033 (0.047)	Data 0.000 (0.008)	Loss 0.8661 (0.9197)	
Epoch: [487][200/688]	Time 0.033 (0.046)	Data 0.000 (0.006)	Loss 0.9836 (0.9193)	
Epoch: [487][250/688]	Time 0.033 (0.044)	Data 0.000 (0.005)	Loss 0.8771 (0.9222)	
Epoch: [487][300/688]	Time 0.033 (0.043)	Data 0.000 (0.004)	Loss 0.9881 (0.9263)	
Epoch: [487][350/688]	Time 0.033 (0.043)	Data 0.000 (0.003)	Loss 0.7912 (0.9225)	
Epoch: [487][400/688]	Time 0.033 (0.043)	Data 0.000 (0.003)	Loss 1.0193 (0.9237)	
Epoch: [487][450/688]	Time 0.033 (0.043)	Data 0.000 (0.003)	Loss 0.9287 (0.9214)	
Epoch: [487][500/688]	Time 0.033 (0.042)	Data 0.000 (0.002)	Loss 0.9864 (0.9195)	
Epoch: [487][550/688]	Time 0.033 (0.042)	Data 0.000 (0.002)	Loss 0.9278 (0.9215)	
Epoch: [487][600/688]	Time 0.033 (0.042)	Data 0.000 (0.002)	Loss 0.8282 (0.9221)	
Epoch: [487][650/688]	Time 0.033 (0.042)	Data 0.000 (0.002)	Loss 0.7567 (0.9224)	
valid_loader typ

Epoch: [492][200/688]	Time 0.033 (0.046)	Data 0.000 (0.006)	Loss 0.9223 (0.9283)	
Epoch: [492][250/688]	Time 0.033 (0.043)	Data 0.000 (0.005)	Loss 1.0417 (0.9242)	
Epoch: [492][300/688]	Time 0.033 (0.043)	Data 0.000 (0.004)	Loss 0.8953 (0.9249)	
Epoch: [492][350/688]	Time 0.033 (0.043)	Data 0.000 (0.003)	Loss 0.8131 (0.9236)	
Epoch: [492][400/688]	Time 0.033 (0.043)	Data 0.000 (0.003)	Loss 0.7935 (0.9231)	
Epoch: [492][450/688]	Time 0.033 (0.043)	Data 0.000 (0.003)	Loss 0.7901 (0.9205)	
Epoch: [492][500/688]	Time 0.033 (0.042)	Data 0.000 (0.002)	Loss 0.8648 (0.9200)	
Epoch: [492][550/688]	Time 0.033 (0.042)	Data 0.000 (0.002)	Loss 0.8008 (0.9199)	
Epoch: [492][600/688]	Time 0.034 (0.042)	Data 0.000 (0.002)	Loss 0.8168 (0.9196)	
Epoch: [492][650/688]	Time 0.033 (0.042)	Data 0.000 (0.002)	Loss 0.7259 (0.9195)	
valid_loader type <class 'torch.utils.data.dataloader.DataLoader'>
valid_loader len: 157
Test: [0/157]	Time 0.373 (0.373)	Loss 0.6232 (0.6232)	Prec@1 82.812 (82.812)
Test: [50/157]

Epoch: [497][300/688]	Time 0.033 (0.043)	Data 0.000 (0.004)	Loss 0.8961 (0.9214)	
Epoch: [497][350/688]	Time 0.033 (0.043)	Data 0.000 (0.003)	Loss 0.9477 (0.9213)	
Epoch: [497][400/688]	Time 0.033 (0.043)	Data 0.000 (0.003)	Loss 0.9340 (0.9206)	
Epoch: [497][450/688]	Time 0.034 (0.043)	Data 0.000 (0.003)	Loss 1.0053 (0.9225)	
Epoch: [497][500/688]	Time 0.033 (0.042)	Data 0.000 (0.002)	Loss 0.9254 (0.9236)	
Epoch: [497][550/688]	Time 0.033 (0.042)	Data 0.000 (0.002)	Loss 0.9770 (0.9241)	
Epoch: [497][600/688]	Time 0.033 (0.042)	Data 0.000 (0.002)	Loss 0.8849 (0.9244)	
Epoch: [497][650/688]	Time 0.033 (0.042)	Data 0.000 (0.002)	Loss 0.8586 (0.9237)	
valid_loader type <class 'torch.utils.data.dataloader.DataLoader'>
valid_loader len: 157
Test: [0/157]	Time 0.397 (0.397)	Loss 0.6136 (0.6136)	Prec@1 81.250 (81.250)
Test: [50/157]	Time 0.004 (0.012)	Loss 0.7195 (0.7190)	Prec@1 70.312 (78.156)
Test: [100/157]	Time 0.004 (0.008)	Loss 0.6362 (0.7058)	Prec@1 78.125 (78.574)
Test: [150/157]	Time 